In [1]:
from box_extractor import BoxExtractor, find_boxes
from cv2 import VideoCapture
import cv2 as cv

extractor = BoxExtractor(find_boxes, slice_size=400, object_size=150)

cap = VideoCapture("worms.avi")

i = 0
while True:
    ret, image = cap.read()
    if ret == False:
        break

    if cv.waitKey(1) & 0xFF == ord("q"):
        break

    coords, rois = extractor.run(image)

    # Draw bboxes
    for box in coords:
        x, y, w, h = box
        cv.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 3)

    cv.imshow("BBOXES", image)

    i += 1

cap.release()
cv.destroyAllWindows()

In [333]:
cv.destroyAllWindows()